# AIM OF THE NOTEBOOK
Plot the results obtained from previous steps in a nice format, with spatial maps that contain the ROIs desired and polygons outlining cells

In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')
import squidpy as sq
import scanpy as sc
import os 
import seaborn as sns
import networkx as nx
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import numpy as np
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import skimage.color
from scipy.sparse import coo_matrix
from scipy.sparse import load_npz
from skimage.measure import label, regionprops
#import scanpy as sc 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.color
import matplotlib.pyplot as plt
from scipy.sparse import load_npz
from scipy.sparse import coo_matrix
import matplotlib as mpl
import tifffile as tff
from matplotlib.pyplot import rc_context
from skimage.segmentation import watershed, expand_labels
import scipy.sparse as sparse
import ast

In [133]:
def make_polygons_from_seg_mask(coo_file,
                               save_path,
                               ):
    ## IMPORT PACKAGES ##
    from skimage.measure import label, regionprops, regionprops_table
    from skimage.morphology import convex_hull_image
    from shapely.geometry import Polygon, mapping, MultiPoint
    from tqdm import tqdm
    import pandas as pd
    import matplotlib.pyplot as plt
    import scanpy as sc
    from scipy.sparse import load_npz
    
    ## LOAD NPZ FILE ##
    coo = load_npz(coo_file)
    
    
    nuclei_list = {}
    array=coo.toarray()
    regions_nuclei = regionprops(array)
    for  i in tqdm(regions_nuclei):
        try:
            mpt = MultiPoint(i.coords)
            #nuclei_list.append(mpt.convex_hull)
            nuclei_list[i.label] = mpt.convex_hull
        except ValueError: 
            #nuclei_list.append(0)
            nuclei_list[i.label] = 0
    cell_id = []
    x_cords = []
    y_cords = []
    for i in nuclei_list:
        try: 
            try: 
                plt.fill(list(nuclei_list[i].exterior.coords.xy[0]),list(nuclei_list[i].exterior.coords.xy[1]), edgecolor='black',linewidth=0.1) #,facecolor=array_sub.iloc[0,3] ,
                cell_id.append(i)
                x_cords.append(list(nuclei_list[i].exterior.coords.xy[0]))
                y_cords.append(list(nuclei_list[i].exterior.coords.xy[1]))

            except KeyError:
                continue
        except AttributeError:
            continue
    plt.axis('scaled')
    plt.show()
    fd = pd.DataFrame(cell_id)
    fd['x_exterior'] = x_cords
    fd['y_exterior'] = y_cords
    #fd.to_csv(save_path)
    return fd

In [134]:
global_path='/media/sergio/PD_DALI/GBM/Processing/NH172680_real/preprocessing/Stitched2DTiles'
coo_file=global_path+'/stardist_segmentation_labels_exp.npz'

In [135]:
sample='NH172680'

In [136]:
#coo_file = path+'stitched/stardist_segmentation_labels_exp.npz'
coo = load_npz(coo_file)
print('The image has %d cells' % len(set(coo.data)))
coo_array = coo.toarray()

The image has 145409 cells


In [137]:
general_path='/media/sergio/GBX/GBmap samples'
#global_path=general_path+'ISS_samples/'
adata=sc.read(general_path+'/ISS_global_analysis/all_samples_with_colors.h5ad') # should be with colors

In [138]:
adatas=adata[adata.obs['sample']==sample]

In [ ]:
pl=make_polygons_from_seg_mask(coo_file,'')

In [141]:
def plot_segmentation_polygon(adata,dapi,
                              polygons,
                              region_subset,
                              cell_subset,
                              color,
                              annotation,
                              save,
                              save_path,
                              cell_id_col = 'CellID',
                             ):
    
    import pandas as pd
    
    
#    polygons = pd.read_csv(polygon_file)
    try:
        polygons = polygons.set_index(0)
    except:
        polygons = polygons.set_index('0')
    adata.obs[cell_id_col] = adata.obs[cell_id_col].astype(int)
    if len(region_subset) == 4: 
        adata = adata[(adata.obs.x>region_subset[0]) & (adata.obs.x<region_subset[1]) & (adata.obs.y>region_subset[2]) & (adata.obs.y<region_subset[3])] 
        
    subset = adata[adata.obs[cell_id_col].isin(pd.Series(polygons.index).astype(int).to_list())]
    
    if len(cell_subset) != 0:
        subset_filt = subset[subset.obs[annotation].isin(cell_subset) ] 
        subset_removed = subset[~subset.obs[annotation].isin(cell_subset) ] 
        subset_removed.obs[color] = '#cecece'
        subset = sc.concat([subset_filt, subset_removed])

    color_dict =  dict(zip(subset.obs[cell_id_col],subset.obs[color]))
    fig=plt.figure(figsize=(15,15),dpi=800)
    
    ax = fig.add_subplot(1, 1, 1)
    with plt.rc_context({'figure.figsize': (15, 15), 'figure.dpi':800}):
        plt.imshow(dapi,cmap='gray',vmin=300,vmax=3300)
        new_list = []
        for i in polygons.index:
            try: 
                try: 
                    plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                             ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                             facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.8) #,facecolor=array_sub.iloc[0,3] ,
                    new_list.append(i)
                except KeyError:
                    continue
                    print('something failed')
            except AttributeError:
                continue
        if len(region_subset) == 4: 
            roi = Rectangle((region_subset[2],region_subset[0]),region_subset[3]-region_subset[2],region_subset[1]-region_subset[0],linewidth=1,edgecolor='black',facecolor='none')
            ax.add_patch(roi);

        plt.axis('scaled')
        plt.rcParams['pdf.fonttype'] = 42
        plt.rcParams['ps.fonttype'] = 42
        plt.rcParams['svg.fonttype'] = 'none'
        if save: 
            plt.savefig(save_path, bbox_inches="tight")
        
        plt.show()

In [142]:
pl.to_csv(global_path+'/polygons_2.csv')

In [143]:
pl=pd.read_csv(global_path+'/polygons_2.csv')

In [144]:
cls=dict(zip(np.unique(adatas.obs['level4']),adatas.uns['level4_colors']))

In [145]:
adatas.obs['color']=adatas.obs['level4'].map(cls)

In [146]:
dapipath=global_path+'/Base_1_stitched-1.tif'

In [147]:
import tifffile as tf
dapi=tf.imread(dapipath)

In [148]:
dapit=dapi.transpose()

# Translate Cell_Num to unique IDs

In [149]:
adatas.obs['Cell_ID']=adatas.obs.index

In [150]:
adatas.obs['subset']=[a[1] for a in adatas.obs.index.str.split('_')]

In [151]:
g1=np.unique(coo_array[:,:10000])
g2=np.unique(coo_array[:,10000:20000])
g3=np.unique(coo_array[:,20000:])

In [152]:
g1ind=[e for e in range(1,len(g1[1:]))]
g2ind=[e for e in range(1,len(g2[1:]))]
g3ind=[e for e in range(1,len(g3[1:]))]

In [153]:
dictio1=dict(zip(g1ind,g1[1:]))
dictio2=dict(zip(g2ind,g2[1:]))
dictio3=dict(zip(g3ind,g3[1:]))

In [154]:
adatas.obs['newid']=adatas.obs['Cell_Num']
adatas.obs.loc[adatas.obs['subset']=='1','newid']=adatas.obs.loc[adatas.obs['subset']=='1','newid'].map(dictio1)
adatas.obs.loc[adatas.obs['subset']=='2','newid']=adatas.obs.loc[adatas.obs['subset']=='2','newid'].map(dictio2)
adatas.obs.loc[adatas.obs['subset']=='3','newid']=adatas.obs.loc[adatas.obs['subset']=='3','newid'].map(dictio3)

In [ ]:
plot_segmentation_polygon(adatas,dapit,pl,list([]),list([]),'color', list(['']),'save',global_path+'/celltype_map_2.pdf',cell_id_col = 'newid')

In [155]:
def plot_segmentation_polygon_nodapi(adata,dapi,
                              polygons,
                              region_subset,
                              cell_subset,
                              color,
                              annotation,
                              save,
                              save_path,
                              cell_id_col = 'CellID',
                             ):
    
    import pandas as pd
    
    
#    polygons = pd.read_csv(polygon_file)
    try:
        polygons = polygons.set_index(0)
    except:
        polygons = polygons.set_index('0')
    adata.obs[cell_id_col] = adata.obs[cell_id_col].astype(int)
    if len(region_subset) == 4: 
        adata = adata[(adata.obs.x>region_subset[0]) & (adata.obs.x<region_subset[1]) & (adata.obs.y>region_subset[2]) & (adata.obs.y<region_subset[3])] 
        
    subset = adata[adata.obs[cell_id_col].isin(pd.Series(polygons.index).astype(int).to_list())]
    
    if len(cell_subset) != 0:
        subset_filt = subset[subset.obs[annotation].isin(cell_subset) ] 
        subset_removed = subset[~subset.obs[annotation].isin(cell_subset) ] 
        subset_removed.obs[color] = '#cecece'
        subset = sc.concat([subset_filt, subset_removed])

    color_dict =  dict(zip(subset.obs[cell_id_col],subset.obs[color]))
    fig=plt.figure(figsize=(15,15),dpi=800)
    
    ax = fig.add_subplot(1, 1, 1)
    with plt.rc_context({'figure.figsize': (15, 15), 'figure.dpi':800}):
       # plt.imshow(dapi,cmap='gray',vmin=300,vmax=1500)
        new_list = []
        for i in polygons.index:
            try: 
                try: 
                    plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                             ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                             facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.8) #,facecolor=array_sub.iloc[0,3] ,
                    new_list.append(i)
                except KeyError:
                    continue
            except AttributeError:
                continue
        if len(region_subset) == 4: 
            roi = Rectangle((region_subset[2],region_subset[0]),region_subset[3]-region_subset[2],region_subset[1]-region_subset[0],linewidth=1,edgecolor='black',facecolor='none')
            ax.add_patch(roi);

        plt.axis('scaled')
        plt.rcParams['pdf.fonttype'] = 42
        plt.rcParams['ps.fonttype'] = 42
        plt.rcParams['svg.fonttype'] = 'none'
   #     if save: 
   #         plt.savefig(save_path, bbox_inches="tight")
        
        plt.show()

# Plot segmentation_arround_certain_celltype

In [161]:
ctp=['Tip-like']
numb=450
rangep=600

In [ ]:
for numb in range(10,200,20):
    print(numb)
    adatact=adatas[adatas.obs['ClassName'].isin(ctp)]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons_2.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]] in ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/tip_like_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [171]:
ctp=['NPC-like Prolif','NPC-like OPC','NPC-like neural','AC-like Prolif','OPC-like Prolif','OPC']
numb=450
rangep=600

In [ ]:
for numb in range(180,500,30):
    print(numb)
    adatact=adatas[adatas.obs['ClassName'].isin(ctp)]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons_2.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]] in ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/niche_neural_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [185]:
ctp=['AC-like','Astrocyte','MES-like hypoxia independent','TAM-BDM anti-infl','VLMC','Scavenging endothelial','Endo capilar']
numb=450
rangep=600

In [ ]:
for numb in range(30,500,40):
    print(numb)
    adatact=adatas[adatas.obs['ClassName'].isin(ctp)]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons_2.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]] in ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/niche_astrocytic_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [190]:
ctp=['Endo arterial','SMC','SMC prolif','Tip-like','Pericyte','SMC COL','Scavenging pericyte']
numb=450
rangep=600

In [ ]:
for numb in range(28,500,40):
    print(numb)
    adatact=adatas[adatas.obs['ClassName'].isin(ctp)]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons_2.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]] in ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/niche_vessel_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [193]:
ctp=['TAM-BDM hypoxia/MES','MES-like hypoxia/MHC','Neutrophil','SMC COL','Mono hypoxia','cDC1']
numb=450
rangep=600

In [ ]:
for numb in range(21,500,20):
    print(numb)
    adatact=adatas[adatas.obs['ClassName'].isin(ctp)]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons_2.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]] in ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/niche_hipoxia_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [102]:
global_path

'/media/sergio/PD_DALI/GBM/Processing/NH172680_real/preprocessing/Stitched2DTiles'

In [ ]:
ctp=['NPC-like Prolif','NPC-like OPC','NPC-like Prolif','NPC-like neural','AC-like Prolif']
numb=450
rangep=700
for numb in range(120,500,20):
    print(numb)
    adatact=adatas[adatas.obs['ClassName']==ctp[0]]
    print(adatact.shape)
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]].isin(ctp):
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0.55)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/NPC_niche_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [ ]:
ctp='AC-like'
numb=450
rangep=700
for numb in range(21,400,10):
    print(numb)
    adatact=adatas[adatas.obs['ClassName']==ctp]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]]==ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.03,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=1)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/AC_like_prolif_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [ ]:
ctp='SMC'
numb=450
rangep=700
for numb in range(27,400,10):
    print(numb)
    adatact=adatas[adatas.obs['ClassName']==ctp]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]]==ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.03,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=1)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/SMC_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [ ]:
ctp='Neutrophil'
numb=450
rangep=700
for numb in range(12,400,15):
    print(numb)
    adatact=adatas[adatas.obs['ClassName']==ctp]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]]==ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.03,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=1)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/Neutrophil_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [ ]:
ctp='MES-like hypoxia/MHC'
numb=450
rangep=700
for numb in range(100,400,10):
    print(numb)
    adatact=adatas[adatas.obs['ClassName']==ctp]
    Xpos=adatact.obs.loc[adatact.obs.index[numb],'X']
    Ypos=adatact.obs.loc[adatact.obs.index[numb],'Y']
    adatas.obs.x=adatas.obs.X
    adatas.obs.y=adatas.obs.Y
    dapit2=dapit[int((Xpos-rangep)):int(Xpos+rangep),int(Ypos-rangep):int(Ypos+rangep)]
    adsub=adatas[adatas.obs['X']>Xpos-rangep]
    adsub=adsub[adsub.obs['X']<Xpos+rangep]
    adsub=adsub[adsub.obs['Y']>Ypos-rangep]
    adsub=adsub[adsub.obs['Y']<Ypos+rangep]
    adsub.obs['Y']=adsub.obs['Y']-(Ypos-rangep)
    adsub.obs['X']=adsub.obs['X']-(Xpos-rangep)
    pl=pd.read_csv(global_path+'/polygons.csv',index_col=1)
    plsub=pl.loc[adsub.obs['newid'],:]
    allx=[]
    ally=[]
    for ii in plsub.index:
        xs=plsub.loc[ii,'x_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
    #    plsub.loc[ii,'x_exterior']=xs
        polx=[]
        for x in range(0,len(xs)):
            polx.append(int(xs[x])-(Ypos-rangep))
        allx.append(polx)
    #    plsub.loc[ii,'x_exterior']=list(xs-(Xpos-rangep))
        xs=plsub.loc[ii,'y_exterior']
        xs=xs.replace('[','').replace(']','').replace('.0','').split(',')
        poly=[]
        for x in range(0,len(xs)):
            poly.append(int(xs[x])-(Xpos-rangep))
        ally.append(poly)
    plsub['x_exterior']=allx
    plsub['y_exterior']=ally
    plsub.columns=[0,'x_exterior','y_exterior']
    polygons=plsub
    subset=adsub
    color_dict =  dict(zip(subset.obs['newid'],subset.obs['color']))
    coldi=dict(zip(adatas.uns['level4_colors'],np.unique(adatas.obs['level4'])))
    general_pathy='/media/sergio/GBX/GBmap samples/'
    global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
    files=os.listdir(global_pathy)
    f=files[9]
    reads=pd.read_csv(global_pathy+f)
    reads=reads[~reads['target'].isna()]
    plt.style.use('dark_background')
    plt.figure(figsize=(6,6),dpi=500)
   # plt.imshow(dapit2,cmap='gray')
    new_list = []
    for i in polygons.index:
                try: 
                    try: 
                        if coldi[color_dict[i]]==ctp:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor='white',linewidth=0.03,alpha=1)
                        else:
                            plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                     ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                     facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=1)#,facecolor=array_sub.iloc[0,3] ,
                        new_list.append(i)
                    except KeyError:
                        continue
                except AttributeError:
                    continue
    plt.axis('off')
    
    plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/MES-like hypoxia_roi1_NH80_'+str(numb)+'.pdf')
    plt.show()

In [91]:
general_pathy='/media/sergio/GBX/GBmap samples/'
global_pathy=general_pathy+'ISS_analysis_10samples/reads/'
files=os.listdir(global_pathy)
f=files[9]
reads=pd.read_csv(global_pathy+f)
reads=reads[~reads['target'].isna()]

In [444]:
selread=['VWF','HSPG2','CLDN5']
reads2=reads[reads['xc']>Xpos-rangep]
reads2=reads2[reads2['xc']<Xpos+rangep]
reads2=reads2[reads2['yc']>Ypos-rangep]
reads2=reads2[reads2['yc']<Ypos+rangep]
reads2['yc']=reads2['yc']-(Ypos-rangep)
reads2['xc']=reads2['xc']-(Xpos-rangep)
reads2=reads2[reads2.target.isin(selread)]

In [445]:
colors_hex = [ "#FFA07A", "#20B2AA", "#9932CC", "#8A2BE2", "#FF4500", "#2E8B57",
    "#6A5ACD", "#D2691E", "#7FFF00", "#DC143C", "#00CED1", "#FFD700",
    "#4B0082", "#32CD32", "#FF6347", "#F0E68C", "#BA55D3", "#ADFF2F",
    "#FF69B4", "#FF8C00", "#FFA500", "#FFDAB9", "#CD5C5C", "#FFA07A",]
dictiocol=dict(zip(np.unique(reads2['target']),colors_hex))

In [ ]:

XDIST=np.max(reads2['xc'])-np.min(reads2['xc'])
YDIST=np.max(reads2['yc'])-np.min(reads2['yc'])
colorlist=reads2['target'].map(dictiocol)
colorlist[colorlist.isna()]='black'
plt.figure(figsize=(XDIST/100,YDIST/100),facecolor='black',dpi=300)
plt.imshow(dapit2,cmap='gray')
new_list = []
for i in polygons.index:
            try: 
                try: 
                    if coldi[color_dict[i]]=='Tip-like':
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=1)
                    else:
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=0)#,facecolor=array_sub.iloc[0,3] ,
                    new_list.append(i)
                except KeyError:
                    continue
            except AttributeError:
                continue



plt.scatter(reads2['yc'],reads2['xc'],c=list(colorlist),s=15)
# Remove axis
#plt.axis('off')
plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/tip_like_roi1_NH80_'+str(numb)+'_with_markers.pdf')

In [ ]:
probs=adsub.obsm['probabilities']['Tip-like']
plt.figure(figsize=(6,6),dpi=500)
plt.imshow(dapit2,cmap='gray')
new_list = []
nume=0
for i in polygons.index:
            try: 
                try: 
                    if coldi[color_dict[i]]=='Tip-like':
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=probs[nume])
                    else:
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=probs[nume])#,facecolor=array_sub.iloc[0,3] ,
                    new_list.append(i)
                except KeyError:
                    continue
            except AttributeError:
                continue
            nume=nume+1
plt.axis('off')
plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/tip_like_roi1_NH80_'+str(numb)+'_probabilities.pdf')

In [ ]:
probs=adsub.obsm['probabilities']['Tip-like']
plt.figure(figsize=(6,6),dpi=500)
plt.imshow(dapit2,cmap='gray')
new_list = []
nume=0
for i in polygons.index:
            try: 
                try: 
                    if coldi[color_dict[i]]=='Tip-like':
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor='white',linewidth=0.7,alpha=probs[nume])
                    else:
                        plt.fill(ast.literal_eval(str(polygons[polygons.index == (i)]['x_exterior'][i])),
                                 ast.literal_eval(str(polygons[polygons.index == (i)]['y_exterior'][i])), 
                                 facecolor = color_dict[i], edgecolor=color_dict[i],linewidth=0.03,alpha=probs[nume])#,facecolor=array_sub.iloc[0,3] ,
                    new_list.append(i)
                except KeyError:
                    continue
            except AttributeError:
                continue
            nume=nume+1
plt.axis('off')
plt.scatter(reads2['yc'],reads2['xc'],c=list(colorlist),s=4)
plt.savefig(general_pathy+'ISS_analysis_10samples/overview_figures/tip_like_roi1_NH80_'+str(numb)+'_probabilities_and_genes.pdf')